- [matching](##match(1))

In [319]:
import pandas as pd
import json

In [320]:
fwg=pd.read_feather("fwg.feather")
df=pd.read_csv("result.csv")
m13=df.columns[1:-2]
df[m13]=df[m13].fillna("").astype(str)
qidLT13=df[df.mngitems<13].QuestID.to_list()
qidAll=df.QuestID.to_list()
fwg=fwg[fwg.QuestID.isin(qidLT13)]
fwg.loc[:,'QLabel']=fwg.loc[:,'QLabel'].str.lower()
fwg.Finding=fwg.Finding.fillna("")
with open("map.json", "r") as j:
    hint=dict(json.loads(j.read()))

In [321]:
def kwSearch(kw:str):
    global fwg
    return fwg[fwg.QLabel.str.contains(kw.lower())]

def assign(qid, item, val):
    '''assign(y-axis,x-axis,val)'''
    global df
    assert (item:=item.lower().strip()) not in ["mngsum13items","mngitems"], f"use `update()` for {item} column."
    df.loc[df.QuestID==qid, item]=val

def match(qlist:list, item:str ,supress=True):
    global fwg,df
    already=[]
    matched=[]
    tempRaw=fwg[fwg.QLabel.isin(qlist)]
    tempQID=tempRaw.QuestID.unique()

    for i in tempQID:
        if i in qidLT13:
            if df.loc[df.QuestID==i, item].values[0]!="":
                already.append(i)
            else:
                            # if all 1 and 0 make it int(...) ↓
                assign(i,item,tempRaw.loc[tempRaw.QuestID==i,"Finding"].values[0])
                matched.append(i)

    del tempRaw

    if supress==False:
        print(f"{len(matched)}/{len(tempQID)} successfully assigned.\n{len(already)}/{len(tempQID)} matched but already had value.")
        return matched
    
def parseUncertain(strng):
    strng=strng.lower().strip()
    if "could not verify" not in strng:
        return strng
    else:
        if "no" not in strng or "yes" not in strng:
            return strng
        else:
            return "1" if strng.split("assessment:")[1].split(".")[0].strip()=="yes" else "0"

In [322]:
for i in hint:
    print(f"\n{i}")
    for kw in hint[i]:
        temp=kwSearch(kw)
        print(f"└╴ {kw}: {len(temp)}")
        if len(temp)<1100:
            print("|",*temp.QLabel.unique(),sep="\n")


hrsigned
└╴ signed: 1551
└╴ hr policy: 1063
|
is there an hr policy that is signed by top management with a clear commitment to meet all legal requirements?
is there an hr policy that is signed by top management?
is there an adequate hr policy that is signed by top management?
is there an hr policy that is signed by top management with a clear commitment to meet all legal requirements, including at least:
- mandatory minimum age requirements
- appropriate work for young workers
- equal treatment and non-discrimination 
- preventing sexual harassment
- legal requirements in relation to termination
- freedom of association,
- working hours and compensation.

in addition, factory policies should include a commitment to resolve workplace problems, including having a grievance handling procedure.
is there an adequate hr policy that is signed by top management with a clear commitment to meet all legal requirements?

recruit
└╴ recruitment: 966
|
does the employer has a recruitment procedure

## match(1)

### problems:
1. findings have `None` (first replace with empty string, process later)
2. findings different values (19541) &rarr; first match, then process special values
3. ...

In [323]:
#hrsigned: hr policy
kwSearch("hr policy").QLabel.unique()
# all:

array(['is there an hr policy that is signed by top management with a clear commitment to meet all legal requirements?',
       'is there an hr policy that is signed by top management?',
       'is there an adequate hr policy that is signed by top management?',
       'is there an hr policy that is signed by top management with a clear commitment to meet all legal requirements, including at least:\n- mandatory minimum age requirements\n- appropriate work for young workers\n- equal treatment and non-discrimination \n- preventing sexual harassment\n- legal requirements in relation to termination\n- freedom of association,\n- working hours and compensation.\n\nin addition, factory policies should include a commitment to resolve workplace problems, including having a grievance handling procedure.',
       'is there an adequate hr policy that is signed by top management with a clear commitment to meet all legal requirements?'],
      dtype=object)

In [324]:
# for i in fwg[fwg.QLabel=='is there an hr policy that is signed by top management?'].QuestID.unique():
#     if df.loc[df["QuestID"]==i,"hrsigned"].isna().values[0]:
#         print(i)
match(kwSearch("hr policy").QLabel.unique(),
      "hrsigned",supress=False)

65/1062 successfully assigned.
997/1062 matched but already had value.


[20649,
 39870,
 39884,
 39949,
 39976,
 40007,
 40009,
 40022,
 40086,
 40166,
 40196,
 40254,
 40274,
 40281,
 40251,
 40285,
 40323,
 40324,
 40340,
 40373,
 40414,
 40525,
 40612,
 40664,
 40622,
 40670,
 40703,
 40791,
 40799,
 40841,
 40892,
 41058,
 41059,
 41178,
 41281,
 41304,
 41312,
 41377,
 41283,
 41408,
 41392,
 41447,
 41469,
 41535,
 41534,
 41547,
 41577,
 41581,
 42087,
 42120,
 42277,
 42355,
 42346,
 42388,
 42473,
 42625,
 42611,
 42854,
 43130,
 43287,
 43366,
 43371,
 43479,
 43566,
 43683]

In [325]:
fwg[(fwg.QuestID==43479) & (fwg.QLabel.isin(kwSearch("hr policy").QLabel.unique()))]

,QuestID,Cluster,QLabel,Finding,FindingText
387341,43479,Contracts and Human Resources,is there an adequate hr policy that is signed ...,"During this visit, BFC decided to carry out a ...",None


In [326]:
df.hrsigned.unique()

array(['During this visit, BFC decided to carry out a more focused risk-based assessment visit, thus this question is not assessed at this time.',
       'During this visit, BFC decided to carry out a more focused risk-based assessment visit, thus this question is not assessed at this time. BFC will follow up on this issue in the next assessment visit.',
       '',
       'During this VCC visit, BFC decided to carry out a more focused risk-based assessment visit, thus this question is not assessed at this time.',
       'During this visit, BFC decided to carry out a more focused risk-based assessment visit, thus this question is not assessed at this time. BFC will keep the information from last assessment visit remain the same as below:Yes. The employer have an HR policy which includes:(a) mandatory minimum age requirements;(b) appropriate work for young workers;(c) equal treatment and non-discrimination;(d) preventing sexual harassment;(e) legal requirements in relation to termination

In [327]:
# recruit: recruitment
kwSearch("recruitment").QLabel.value_counts()
# all except last one

does the employer have an adequate recruitment procedure?                                                                                                                                                                                                                                                                                                                                                             771
does the employer have a recruitment procedure?                                                                                                                                                                                                                                                                                                                                                                       125
does the employer has a recruitment procedure?                                                                                                                                      

In [328]:
match(kwSearch("recruitment").QLabel.unique()[:-1],
      m13[1],supress=False)

414/962 successfully assigned.
548/962 matched but already had value.


[20649,
 39709,
 39760,
 39758,
 39759,
 39820,
 39803,
 39837,
 39869,
 39870,
 39852,
 39887,
 39884,
 39902,
 39906,
 39936,
 39949,
 39965,
 39966,
 39981,
 39976,
 40007,
 40009,
 40022,
 40025,
 40024,
 40057,
 40074,
 40086,
 40091,
 40125,
 40141,
 40153,
 40154,
 40166,
 40186,
 40193,
 40196,
 40242,
 40254,
 40267,
 40274,
 40280,
 40281,
 40284,
 40251,
 40232,
 40272,
 40285,
 40286,
 40323,
 40324,
 40325,
 40339,
 40343,
 40340,
 40337,
 40373,
 40372,
 40379,
 40399,
 40407,
 40402,
 40401,
 40412,
 40426,
 40414,
 40442,
 40456,
 40468,
 40469,
 40476,
 40470,
 40496,
 40528,
 40523,
 40525,
 40545,
 40546,
 40550,
 40524,
 40559,
 40589,
 40591,
 40600,
 40595,
 40612,
 40616,
 40593,
 40620,
 40619,
 40621,
 40629,
 40626,
 40651,
 40652,
 40662,
 40663,
 40664,
 40622,
 40683,
 40670,
 40684,
 40689,
 40688,
 40696,
 40703,
 40705,
 40714,
 40715,
 40717,
 40723,
 40727,
 40728,
 40733,
 40729,
 40738,
 40749,
 40748,
 40752,
 40771,
 40776,
 40775,
 40787,
 40781,


In [329]:
# discfire: disciplinary and termination
kwSearch("disciplinary and termination").QLabel.value_counts()
# all

does the employer have adequate disciplinary and termination procedures?                                                                                                                                                                                                                                                                             646
does the employer have disciplinary and termination procedures?                                                                                                                                                                                                                                                                                      249
does the employer has disciplinary and termination procedures?                                                                                                                                                                                                                                                        

In [330]:
match(kwSearch("disciplinary and termination").QLabel.unique(),
      m13[2],supress=False)

387/961 successfully assigned.
574/961 matched but already had value.


[20649,
 33503,
 39709,
 39760,
 39758,
 39820,
 39803,
 39837,
 39869,
 39870,
 39852,
 39884,
 39902,
 39906,
 39936,
 39949,
 39965,
 39966,
 39981,
 39976,
 40007,
 40009,
 40022,
 40025,
 40024,
 40057,
 40074,
 40086,
 40091,
 40125,
 40141,
 40153,
 40154,
 40166,
 40186,
 40193,
 40196,
 40242,
 40254,
 40267,
 40274,
 40280,
 40281,
 40284,
 40251,
 40232,
 40272,
 40285,
 40286,
 40323,
 40324,
 40325,
 40339,
 40343,
 40340,
 40337,
 40373,
 40372,
 40379,
 40399,
 40407,
 40402,
 40401,
 40426,
 40414,
 40442,
 40468,
 40469,
 40470,
 40496,
 40528,
 40523,
 40525,
 40545,
 40524,
 40559,
 40589,
 40591,
 40600,
 40595,
 40612,
 40616,
 40593,
 40620,
 40621,
 40629,
 40626,
 40651,
 40652,
 40662,
 40663,
 40664,
 40622,
 40683,
 40670,
 40684,
 40689,
 40688,
 40696,
 40703,
 40705,
 40714,
 40715,
 40717,
 40723,
 40727,
 40728,
 40733,
 40729,
 40738,
 40749,
 40748,
 40752,
 40771,
 40776,
 40775,
 40787,
 40781,
 40791,
 40799,
 40802,
 40805,
 40841,
 40880,
 40891,


In [331]:
# grievdispute: grievance handling
grg=list(kwSearch("grievance handling").QLabel.unique()[:3])+\
list(kwSearch("grievance handling").QLabel.unique()[4:])
grg
# all except 4 (967-3=964)

['does the employer has grievance handling and dispute resolution procedures?',
 'does the employer have grievance handling and dispute resolution procedures?',
 'does the employer have adequate grievance handling and dispute resolution procedures?',
 'does the employer have grievance handling and dispute resolution procedures that include at least:\n- clear options for submitting grievances and disputes that ensure anonymity and non-retaliation;\n- fair review and appeal process;\n- communication of changes made and resolution as appropriate.']

In [332]:
match(grg, m13[3], supress=False)

165/963 successfully assigned.
798/963 matched but already had value.


[20649,
 33503,
 39758,
 39759,
 39852,
 39884,
 39949,
 39965,
 39981,
 40007,
 40009,
 40022,
 40086,
 40166,
 40193,
 40196,
 40254,
 40274,
 40281,
 40284,
 40251,
 40285,
 40323,
 40324,
 40340,
 40407,
 40412,
 40414,
 40468,
 40545,
 40546,
 40524,
 40589,
 40612,
 40664,
 40622,
 40683,
 40670,
 40684,
 40696,
 40703,
 40723,
 40729,
 40748,
 40787,
 40781,
 40805,
 40841,
 40891,
 40892,
 41008,
 41023,
 41058,
 41059,
 41091,
 41178,
 41187,
 41076,
 41234,
 41185,
 41281,
 41304,
 41316,
 41317,
 41324,
 41355,
 41378,
 41312,
 41283,
 41392,
 41438,
 41447,
 41484,
 41469,
 41471,
 41504,
 41534,
 41539,
 41541,
 41547,
 41566,
 41577,
 41581,
 41586,
 41641,
 42024,
 42087,
 42051,
 42096,
 42212,
 42220,
 42262,
 42275,
 42277,
 42301,
 42306,
 42346,
 42377,
 42388,
 42395,
 42436,
 42435,
 42527,
 42555,
 42558,
 42568,
 42601,
 42615,
 42614,
 42608,
 42632,
 42684,
 42674,
 42740,
 42750,
 42759,
 42863,
 42867,
 42872,
 42916,
 42927,
 42955,
 42957,
 42960,
 43001,


In [333]:
# accounthr: following factory policies and procedures
kwSearch("following factory policies and procedures").QLabel.unique()
# all

array(['does the employer assigns accountability at all levels of management for following factory policies and procedures relating to human resource management and performance?',
       'does the employer assign accountability at all levels of management for following factory policies and procedures relating to human resource management and performance including:\n- reporting lines/organizational charts;\n- description of job duties and authority;\n- accountability through evaluation of performance.',
       'does the employer assign accountability at all levels of management for following factory policies and procedures relating to human resource management and performance?',
       'does the employer adequately assign accountability to management for following factory policies and procedures relating to human resource management and performance?',
       'does the employer adequately assign accountability to management for following factory policies and procedures relating to hr man

In [334]:
match(kwSearch("following factory policies and procedures").QLabel.unique(),
      m13[4], supress=0)

150/486 successfully assigned.
336/486 matched but already had value.


[20649,
 21909,
 27473,
 39803,
 39869,
 39884,
 39906,
 39949,
 39965,
 40007,
 40009,
 40022,
 40024,
 40057,
 40086,
 40125,
 40154,
 40166,
 40186,
 40193,
 40196,
 40254,
 40274,
 40285,
 40286,
 40323,
 40324,
 40325,
 40343,
 40340,
 40373,
 40379,
 40401,
 40426,
 40414,
 40469,
 40496,
 40528,
 40523,
 40546,
 40559,
 40589,
 40591,
 40600,
 40612,
 40616,
 40626,
 40662,
 40664,
 40622,
 40670,
 40703,
 40715,
 40723,
 40733,
 40729,
 40738,
 40841,
 40892,
 40994,
 41015,
 41058,
 41121,
 41123,
 41135,
 41145,
 41143,
 41170,
 41172,
 41076,
 41224,
 41185,
 41239,
 41278,
 41316,
 41324,
 41329,
 41320,
 41379,
 41312,
 41377,
 41283,
 41405,
 41408,
 41271,
 41392,
 41430,
 41422,
 41447,
 41473,
 41527,
 41534,
 41546,
 41547,
 41581,
 41599,
 41617,
 41636,
 41585,
 41876,
 42149,
 42310,
 42372,
 42496,
 42500,
 42527,
 42529,
 42551,
 42553,
 42556,
 42573,
 42604,
 42637,
 42674,
 42717,
 42750,
 42759,
 42785,
 42919,
 42961,
 43001,
 43098,
 43130,
 43163,
 43101,


In [335]:
# comimplhr: implement hr
kwSearch("implement hr").QLabel.unique()
# only 1?

array(['does the employer effectively communicate and implement hr policies and procedures?',
       'does the employer adequately communicate and implement hr policies and procedures?'],
      dtype=object)

In [336]:
match([kwSearch("implement hr").QLabel.unique()],m13[5],0)

0/0 successfully assigned.
0/0 matched but already had value.


[]

In [337]:
# investihr: evaluate performance of hr
kwSearch("evaluate performance of hr").QLabel.unique()
# hr policies and procedures to identify weaknesses

array(['does the employer monitor and evaluate performance of hr policies and procedures?',
       'does the employer adequately monitor and evaluate performance of hr policies and procedures?'],
      dtype=object)

In [338]:
match(kwSearch("hr policies and procedures to identify weaknesses").QLabel.unique(),
      m13[6],0)

413/486 successfully assigned.
73/486 matched but already had value.


[20649,
 33503,
 39709,
 39760,
 39758,
 39759,
 39820,
 39803,
 39837,
 39869,
 39870,
 39852,
 39887,
 39884,
 39902,
 39906,
 39936,
 39949,
 39965,
 39966,
 39981,
 39976,
 40007,
 40009,
 40022,
 40025,
 40024,
 40057,
 40074,
 40086,
 40091,
 40125,
 40141,
 40153,
 40154,
 40166,
 40186,
 40193,
 40196,
 40242,
 40254,
 40267,
 40274,
 40280,
 40281,
 40284,
 40251,
 40232,
 40272,
 40285,
 40286,
 40323,
 40324,
 40325,
 40339,
 40343,
 40340,
 40337,
 40373,
 40372,
 40379,
 40399,
 40407,
 40402,
 40401,
 40412,
 40426,
 40414,
 40442,
 40456,
 40468,
 40469,
 40476,
 40470,
 40496,
 40528,
 40523,
 40525,
 40545,
 40546,
 40550,
 40524,
 40559,
 40589,
 40591,
 40600,
 40595,
 40612,
 40616,
 40593,
 40620,
 40619,
 40621,
 40629,
 40626,
 40651,
 40652,
 40662,
 40663,
 40664,
 40622,
 40683,
 40670,
 40684,
 40689,
 40688,
 40696,
 40703,
 40705,
 40714,
 40715,
 40717,
 40723,
 40727,
 40728,
 40733,
 40729,
 40738,
 40749,
 40748,
 40752,
 40771,
 40776,
 40775,
 40787,


In [339]:
# emergencepr: <both>
kwSearch("emergency preparedness").QLabel.unique()
# all?

array(['is there an emergency preparedness procedure?',
       'is there an adequate  emergency preparedness procedure?',
       'is there an emergency preparedness procedure that includes methods for:\n- reporting fires and other emergencies;\n- alerting all employees to evacuate;\n- evacuating employees to designated assembly location;\n- accounting for all employees after an evacuation.',
       'is there an adequate emergency preparedness procedure?'],
      dtype=object)

In [340]:
match(kwSearch("emergency preparedness").QLabel.unique(),
      m13[7],0)

420/905 successfully assigned.
485/905 matched but already had value.


[20649,
 21902,
 33503,
 39709,
 39760,
 39758,
 39759,
 39820,
 39803,
 39837,
 39869,
 39870,
 39852,
 39887,
 39884,
 39902,
 39906,
 39936,
 39949,
 39965,
 39966,
 39981,
 39976,
 40007,
 40009,
 40022,
 40025,
 40024,
 40057,
 40074,
 40086,
 40091,
 40125,
 40141,
 40153,
 40154,
 40166,
 40186,
 40193,
 40196,
 40242,
 40254,
 40267,
 40274,
 40280,
 40281,
 40284,
 40251,
 40232,
 40272,
 40285,
 40286,
 40323,
 40324,
 40325,
 40339,
 40343,
 40340,
 40337,
 40373,
 40372,
 40379,
 40399,
 40407,
 40402,
 40401,
 40412,
 40426,
 40414,
 40442,
 40456,
 40468,
 40476,
 40470,
 40496,
 40528,
 40523,
 40525,
 40545,
 40546,
 40550,
 40524,
 40559,
 40589,
 40591,
 40600,
 40595,
 40612,
 40616,
 40593,
 40620,
 40619,
 40621,
 40629,
 40626,
 40651,
 40652,
 40662,
 40663,
 40664,
 40622,
 40683,
 40670,
 40684,
 40689,
 40688,
 40696,
 40703,
 40705,
 40714,
 40715,
 40717,
 40723,
 40727,
 40728,
 40733,
 40729,
 40738,
 40749,
 40748,
 40752,
 40771,
 40776,
 40775,
 40787,


In [341]:
# riskcontr: risk management
kwSearch("risk management").QLabel.unique()
# all

array(['is there a hazard/risk management and control procedure?',
       'is there an adequate hazard/risk management and control procedure?',
       'is there a hazard/risk management and control procedure that includes:\n- a systematic approach to identifying hazards;\n- a structure to prioritise risks based on potential impact and likelihood;\n- a hierarchy of controls that is used to select effective controls;\n- a risk register that drives the implementation of controls.'],
      dtype=object)

In [342]:
match(kwSearch("risk management").QLabel.unique(),m13[8],0)

87/902 successfully assigned.
815/902 matched but already had value.


[20649,
 33503,
 39852,
 39887,
 39884,
 39949,
 39965,
 39981,
 40007,
 40009,
 40022,
 40086,
 40153,
 40166,
 40196,
 40254,
 40274,
 40285,
 40323,
 40324,
 40343,
 40340,
 40414,
 40456,
 40470,
 40600,
 40612,
 40619,
 40664,
 40622,
 40703,
 40729,
 40748,
 40787,
 40841,
 40891,
 40892,
 41058,
 41105,
 41185,
 41316,
 41312,
 41283,
 41392,
 41447,
 41473,
 41534,
 41546,
 41547,
 41572,
 41577,
 41581,
 41617,
 42306,
 42344,
 42527,
 42546,
 42674,
 42712,
 42759,
 42981,
 43001,
 43032,
 43130,
 43209,
 43244,
 43253,
 43260,
 43235,
 43287,
 43288,
 43317,
 43143,
 43371,
 43418,
 43450,
 43482,
 43491,
 43476,
 43500,
 43477,
 43566,
 43599,
 43553,
 43684,
 43683,
 43750]

In [343]:
# accidentinvesti: accident investigation
kwSearch("accident investigation").QLabel.unique()
# all

array(['is there an accident investigation procedure?',
       'is there an adequate  accident investigation procedure?',
       'is there an accident investigation procedure that includes:\n- root cause analysis;\n- focuses on sustainable solutions;\n- leads to change to avoid reoccurrence.',
       'is there an accident investigation procedure ?',
       'is there an adequate accident investigation procedure?'],
      dtype=object)

In [344]:
match(kwSearch("accident investigation").QLabel.unique(),
      m13[9],0)

413/902 successfully assigned.
489/902 matched but already had value.


[9617,
 20649,
 33503,
 39709,
 39760,
 39758,
 39759,
 39820,
 39803,
 39837,
 39869,
 39870,
 39852,
 39887,
 39884,
 39902,
 39906,
 39936,
 39949,
 39965,
 39966,
 39981,
 40007,
 40009,
 40022,
 40025,
 40024,
 40057,
 40074,
 40086,
 40091,
 40125,
 40141,
 40153,
 40154,
 40166,
 40186,
 40193,
 40196,
 40242,
 40254,
 40267,
 40274,
 40280,
 40281,
 40284,
 40251,
 40232,
 40272,
 40285,
 40286,
 40323,
 40324,
 40325,
 40339,
 40343,
 40340,
 40337,
 40373,
 40372,
 40379,
 40399,
 40407,
 40402,
 40401,
 40412,
 40426,
 40414,
 40442,
 40456,
 40468,
 40469,
 40476,
 40470,
 40496,
 40528,
 40523,
 40525,
 40545,
 40546,
 40550,
 40524,
 40559,
 40589,
 40591,
 40600,
 40595,
 40612,
 40616,
 40593,
 40620,
 40619,
 40621,
 40629,
 40626,
 40651,
 40652,
 40662,
 40663,
 40664,
 40622,
 40683,
 40670,
 40684,
 40689,
 40688,
 40696,
 40703,
 40705,
 40714,
 40715,
 40717,
 40723,
 40727,
 40728,
 40733,
 40729,
 40738,
 40749,
 40748,
 40752,
 40771,
 40776,
 40775,
 40787,
 

In [345]:
# accohs: for carrying out
kwSearch("for carrying out").QLabel.unique()
#all

array(['does the employer assign accountability at all levels of management for carrying out health and safety responsibilities including:\n- reporting lines/organizational charts;\n- description of job duties and authority;\n- accountability through evaluation of performance;\n- osh officer with expertise and authority to perform responsibilities\n- establishment of a joint osh committee.',
       'does the employer assign accountability at all levels of management for carrying out health and safety responsibilities?',
       'does the employer adequately assign accountability to management for carrying out health and safety responsibilities?',
       'does the employer adequately assign accountability to management for carrying out osh responsibilities?'],
      dtype=object)

In [346]:
match(kwSearch("for carrying out").QLabel.unique(),m13[10],0)

284/426 successfully assigned.
142/426 matched but already had value.


[20649,
 33503,
 39709,
 39760,
 39758,
 39759,
 39820,
 39803,
 39837,
 39869,
 39870,
 39852,
 39887,
 39884,
 39949,
 39966,
 39981,
 40007,
 40009,
 40022,
 40025,
 40074,
 40086,
 40125,
 40153,
 40154,
 40166,
 40186,
 40193,
 40196,
 40242,
 40254,
 40267,
 40274,
 40280,
 40251,
 40232,
 40285,
 40323,
 40324,
 40340,
 40337,
 40379,
 40399,
 40407,
 40401,
 40414,
 40456,
 40469,
 40476,
 40470,
 40523,
 40545,
 40546,
 40524,
 40591,
 40612,
 40616,
 40620,
 40619,
 40621,
 40629,
 40651,
 40652,
 40662,
 40664,
 40622,
 40683,
 40670,
 40684,
 40689,
 40688,
 40696,
 40703,
 40714,
 40717,
 40727,
 40728,
 40733,
 40729,
 40738,
 40749,
 40752,
 40776,
 40775,
 40787,
 40781,
 40805,
 40841,
 40880,
 40892,
 41008,
 41058,
 41059,
 41079,
 41091,
 41105,
 41113,
 41135,
 41145,
 41178,
 41234,
 41185,
 41182,
 41239,
 41281,
 41278,
 41304,
 41316,
 41317,
 41324,
 41355,
 41320,
 41378,
 41323,
 41312,
 41377,
 41382,
 41283,
 41410,
 41411,
 41408,
 41387,
 41271,
 41392,


In [347]:
# impleosh: implement osh
kwSearch("implement osh").QLabel.unique()
# all

array(['does the employer effectively communicate and implement osh policies and procedures?',
       'does the employer adequately communicate and implement osh policies and procedures?'],
      dtype=object)

In [348]:
match(kwSearch("implement osh").QLabel.unique(),m13[11],0)

390/896 successfully assigned.
506/896 matched but already had value.


[9617,
 10515,
 20649,
 33503,
 39709,
 39760,
 39758,
 39759,
 39820,
 39803,
 39837,
 39869,
 39870,
 39852,
 39887,
 39884,
 39902,
 39906,
 39936,
 39949,
 39965,
 39966,
 39981,
 39976,
 40007,
 40009,
 40022,
 40025,
 40024,
 40057,
 40074,
 40086,
 40091,
 40125,
 40153,
 40154,
 40166,
 40186,
 40193,
 40196,
 40242,
 40254,
 40267,
 40274,
 40280,
 40281,
 40284,
 40251,
 40232,
 40272,
 40285,
 40286,
 40323,
 40324,
 40325,
 40339,
 40343,
 40340,
 40337,
 40379,
 40399,
 40407,
 40402,
 40401,
 40412,
 40414,
 40442,
 40456,
 40468,
 40469,
 40476,
 40528,
 40523,
 40525,
 40545,
 40546,
 40550,
 40524,
 40559,
 40589,
 40591,
 40600,
 40595,
 40612,
 40616,
 40620,
 40619,
 40621,
 40629,
 40626,
 40651,
 40652,
 40662,
 40663,
 40664,
 40622,
 40683,
 40684,
 40689,
 40688,
 40696,
 40703,
 40705,
 40714,
 40715,
 40717,
 40727,
 40728,
 40733,
 40729,
 40738,
 40749,
 40748,
 40752,
 40771,
 40776,
 40775,
 40787,
 40781,
 40791,
 40799,
 40802,
 40805,
 40841,
 40880,
 

In [349]:
# invstiosh: measure osh issues
kwSearch("measure osh issues").QLabel.unique()
# all

array(['does the employer investigate, monitor and measure osh issues?',
       'does the employer adequately investigate, monitor and measure osh issues?',
       'does the employer investigate, monitor and measure osh issues to identify root causes and make necessary adjustments to prevent recurrence, including but not limited to: \n- tests, surveys and inspections regularly carried out.\n- violations of procedures logged and analyzed.\n- regular management review of effectiveness of management system including performance on measurable objectives and targets.',
       'does the employer investigate, monitor and measure osh issues to identify root causes and make necessary adjustments to prevent recurrence?',
       'does the employer adequately investigate, monitor and measure osh issues to identify root causes and make necessary adjustments to prevent recurrence?'],
      dtype=object)

In [350]:
match(kwSearch("measure osh issues").QLabel.unique(),
      m13[12],0)

239/897 successfully assigned.
658/897 matched but already had value.


[20649,
 33503,
 39709,
 39760,
 39758,
 39869,
 39870,
 39852,
 39884,
 39902,
 39906,
 39949,
 39966,
 39976,
 40007,
 40009,
 40022,
 40025,
 40074,
 40086,
 40125,
 40141,
 40153,
 40154,
 40166,
 40193,
 40196,
 40242,
 40254,
 40267,
 40274,
 40280,
 40284,
 40251,
 40232,
 40285,
 40323,
 40324,
 40339,
 40343,
 40340,
 40337,
 40402,
 40414,
 40442,
 40468,
 40469,
 40523,
 40525,
 40546,
 40550,
 40524,
 40589,
 40591,
 40595,
 40612,
 40616,
 40620,
 40621,
 40652,
 40664,
 40622,
 40683,
 40684,
 40689,
 40688,
 40703,
 40705,
 40714,
 40727,
 40733,
 40738,
 40752,
 40771,
 40775,
 40781,
 40791,
 40799,
 40802,
 40805,
 40841,
 40892,
 41008,
 41058,
 41061,
 41059,
 41079,
 41113,
 41121,
 41123,
 41135,
 41145,
 41170,
 41178,
 41207,
 41224,
 41219,
 41182,
 41263,
 41281,
 41304,
 41329,
 41355,
 41378,
 41312,
 41377,
 41382,
 41283,
 41410,
 41411,
 41408,
 41387,
 41392,
 41419,
 41460,
 41447,
 41473,
 41484,
 41469,
 41471,
 41504,
 41510,
 41523,
 41535,
 41534,


In [351]:
df.to_csv("result0703.csv",index=False)

## manually check all new entries:
1. could not verify, use old result
2. could not verify &rarr; 0
3. lorem &rarr; NA
4. wierd number (7) &rarr; 1
5. normal answer (need manually interpret) &rarr; 0

In [309]:
oQ=[]
for i in m13:
    oQ.extend([x for x  in df[i].unique()]) #
oQ=list(set(oQ))
oQ

['',
 'During this visit, BFC decided to carry out a more focused risk-based assessment visit, thus this question is not assessed at this time. BFC will keep the information from last assessment visit remain the same as below: No. The employer does not adequately assign accountability to management staff for carrying out health and safety responsibilities. BFC recommends that the employer assigns accountability through:- defined accountability and/or responsibility for OSH issues in writing for the OSH officer(s) as well as themember(s) of top management to whom that person reports;- an OSH officer with expertise and authority to perform relevant responsibilities; and- an OSH committee or similar worker/management committee with a mandate to consider OSH issues in the workplace.',
 'During this visit, BFC decided to carry out a more focused risk-based assessment visit, thus this question is not assessed at this time. BFC will keep the information from last assessment visit remain the s

In [352]:
[x for x in oQ if "during" not in x.lower()]

['',
 '7',
 'Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?',
 'The employer should effectively communicate and implement OSH policies and procedures through:- worker training relev

In [265]:
oQ=[]
for i in m13:
    oQ.extend([x for x  in df[i].unique() if "thus" not in x and "unde" not in x]) #
oQ=[x for x in oQ if x not in ["0.0","1.0",""]]
oQ

['We could not verify this issue during one day visit, so we kept the previous finding and follow-up in the next assessment: Yes. The employer has adequate disciplinary and termination procedures that include a clear explanation of unacceptable behaviour or performance, a step-by-step warning system, the right to defend oneself prior to termination and to representation during disciplinary processes, legal notice periods and termination payments,and timeframes for warnings and payments.',
 'We could not verify this issue during one day visit, so we kept the previous finding and follow-up in the next assessment: No. The employer does not have adequate disciplinary and termination procedures cover (1) legal notice periods and termination payments; and (2) timeframes for warnings and payments. However, employer had internal regulations which include (a) a clear explanation of unacceptable behavior or performance; (b) a step-by-step warning system; and (c) the rights to defend oneself prio

In [286]:
normalNewQ=[
'bfc recommends that the employer ensure that accountability and responsibility for hr issues is defined in writing for the hr officer(s) as well as the member of top management to whom that person reports. this can be verified through: - job descriptions;- reporting lines/organizational charts;- description of job duties and authority; and/or- accountability through evaluation of performance.',
'the employer should set up an accident investigation procedure, including:- root cause analysis;- a focus on sustainable solutions; and- changes aimed at avoiding recurrence.',
'the employer should effectively communicate and implement osh policies and procedures through:- worker training relevant to assigned tasks;- training of the joint osh committee;- posting names of the osh committee members in the workplace;- posting evacuation plans in the workplace; and- ensuring that policies and work instructions are posted and easily accessible in the factory.',
'the employer should first develop and then communicate osh policies and procedures through:- worker training relevant to assigned tasks; - training of a joint osh committee; - posting names of the osh committee members in the workplace; - posting evacuation plans in the workplace; and- posting policies and work instructions that are easily accessible in the workplace.'
]

In [284]:
[parseUncertain(x) for x  in oQ]

['1',
 'we could not verify this issue during one day visit, so we kept the previous finding and follow-up in the next assessment: no. the employer does not have adequate disciplinary and termination procedures cover (1) legal notice periods and termination payments; and (2) timeframes for warnings and payments. however, employer had internal regulations which include (a) a clear explanation of unacceptable behavior or performance; (b) a step-by-step warning system; and (c) the rights to defend oneself prior to termination, and to representation during disciplinary processes.documents checked: internal regulations registered with molvt october 12, 2011.',
 'the assessor could not verify this question because the factory did not provide any documents.',
 '1',
 '1',
 'we could not verify this issue during one day visit, so we kept the previous finding and follow-up in the next assessment: no. the employer does not have adequate disciplinary and termination procedures. bfc recommends that

In [296]:
for i in normalNewQ:
    print(f"{i}:\n{fwg[fwg.Finding.str.lower()==i].index.to_list()}")

bfc recommends that the employer ensure that accountability and responsibility for hr issues is defined in writing for the hr officer(s) as well as the member of top management to whom that person reports. this can be verified through: - job descriptions;- reporting lines/organizational charts;- description of job duties and authority; and/or- accountability through evaluation of performance.:
[158925, 207386]
the employer should set up an accident investigation procedure, including:- root cause analysis;- a focus on sustainable solutions; and- changes aimed at avoiding recurrence.:
[15227]
the employer should effectively communicate and implement osh policies and procedures through:- worker training relevant to assigned tasks;- training of the joint osh committee;- posting names of the osh committee members in the workplace;- posting evacuation plans in the workplace; and- ensuring that policies and work instructions are posted and easily accessible in the factory.:
[15223]
the employ

In [308]:
fwg.loc[15227].QLabel

'is there an accident investigation procedure ?'

In [ ]:
for i in m13:
    